In [2]:
! ln -s /usr/local/courses/lt2318/data/ data

ln: failed to create symbolic link 'data/data': File exists


In [3]:
! ls data/visual_genome/

image_data.json      objects.pickle	       VG_100K
image_data.json.zip  read_objects_resnet50.py  VG_100K_2
images2.zip	     relationships.json
images.zip	     relationships.json.zip


In [4]:
import json
from collections import Counter, defaultdict
import numpy as np
import pickle

%matplotlib inline
from matplotlib import pyplot as plt

In [5]:
# read from file
relationships_from_file = json.load(open('data/visual_genome/relationships.json'))

# name/names correction for reading content of nodes in the dataset
name_extract = lambda x: x['names'][0].lower() if 'names' in x and len(x['names']) else x['name'].lower() if 'name' in x else '' 

In [6]:
print('number of images:', len(relationships_from_file))

number of images: 108077


In [7]:
image_data_from_file = json.load(open('data/visual_genome/image_data.json'))
image_id_to_size = {
    image_data['image_id']: (image_data['width'], image_data['height'])
    for image_data in image_data_from_file
}

In [8]:
# Functions for getting descriptions (relations) and bounding boxes (which are normalised by image size in the process).
def get_description(relations_data):
    triplet = (
        name_extract(relation_data['subject']),
        relation_data['predicate'].lower(), # synset?
        name_extract(relation_data['object']),
    )
    
    return triplet

def get_bboxes(relations_data, size):
    w0, h0 = size
    def normalize(bbox):
        x, y, w, h = bbox
        return (x/w0, y/h0, w/w0, h/h0, )
    
    bboxes = (
        normalize([relation_data['subject'][d] for d in ['x', 'y', 'w', 'h', ]]),
        normalize([relation_data['object'][d] for d in ['x', 'y', 'w', 'h', ]]),
    )
    
    return bboxes

In [9]:
prepositions = ['over', 'above', 'below', 'under', 'on', 'in', 'right of', 'left of']

In [10]:
# Loading visual data.
with open('/usr/local/courses/lt2318/data/visual_genome/objects.pickle', 'rb') as f:
    visual_features = pickle.load(f)

In [11]:
# Function for getting the visual feature vectors from the previous loaded file based on the object_id of the subject and object in a relation.
def get_vis_vecs(relations_data, vis_features):
    subid = relations_data['subject']['object_id']
    objid = relations_data['object']['object_id']
    
    try:
        target_vf = vis_features[subid]
    except KeyError:
        target_vf = None
    try:
        landmark_vf = vis_features[objid]
    except KeyError:
        landmark_vf = None
                
    return target_vf, landmark_vf

In [12]:
# Creating a dictionary which contains another dictionary for each preposition, which contains two lists - one for targets and one for landmarks.
prep_vis_feat = {
    p: {}
    for p in prepositions
}

for p in prep_vis_feat:
    prep_vis_feat[p]['target'] = []
    prep_vis_feat[p]['landmark'] = []

# Gets visual feature vectors for relations if their predicate is one of the prepositions in question.
# Also checks that there are both target and landmark for each relation, and only keeps the ones that have the full pairs.
# This was to deal with the fact that I was initially getting different numbers of targets and landmarks.
for img_relations in relationships_from_file:
    for relation_data in img_relations['relationships']:
        _, p, _ = get_description(relation_data)
        
        if p in prep_vis_feat:
            target_vf, landmark_vf = get_vis_vecs(relation_data, visual_features)
            
            if target_vf is not None and landmark_vf is not None:
                prep_vis_feat[p]['target'].append(target_vf)
                prep_vis_feat[p]['landmark'].append(landmark_vf)
            else:
                pass

In [13]:
for p in prep_vis_feat:
    print(p, len(prep_vis_feat[p]["target"]), len(prep_vis_feat[p]["landmark"]))

over 8200 8200
above 12583 12583
below 3155 3155
under 16985 16985
on 544509 544509
in 183951 183951
right of 220 220
left of 365 365


In [14]:
# Saves the dictionary containing all of the visual feature vectors divided by targets and landmark by preposition to file.
with open('pvis_vecs.pkl', 'wb') as sf:
    pickle.dump(prep_vis_feat, sf)

In [15]:
type(prep_vis_feat[p]["target"])
prep_vis_feat[p]["target"][0].shape

(2048,)

In [18]:
# Takes all target and landmark vectors from prep_vis_feat. Sets a variable to the first vector of each set
# and then adds them together (one addition version and one append version currently). 
from scipy.spatial.distance import cosine

for p in prep_vis_feat:
    targ_vecs = prep_vis_feat[p]['target']
    landm_vecs = prep_vis_feat[p]['landmark']
    
    # creating centroids
    targ_centroid = np.array(targ_vecs).sum(0)
    landm_centroid = np.array(landm_vecs).sum(0)
    
    # cosine distances:
    targ_cos = np.array([cosine(u, targ_centroid) for u in targ_vecs])
    landm_cos = np.array([cosine(u, landm_centroid) for u in landm_vecs])
    
    # average cosine distances (variance):
    targ_var = targ_cos.mean()
    landm_var = landm_cos.mean()
    
    # print or save these numbers
    print(p, "\n", "Target variance:",  targ_var, "Landmark variance:", landm_var)

over 
 Target variance: 0.4377860509985831 Landmark variance: 0.43583857570843
above 
 Target variance: 0.4416074931721027 Landmark variance: 0.4337125514527151
below 
 Target variance: 0.4496400695075702 Landmark variance: 0.45063913436680325
under 
 Target variance: 0.44614292008054934 Landmark variance: 0.44368264179195344


/usr/local/lib64/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


on 
 Target variance: nan Landmark variance: nan
in 
 Target variance: nan Landmark variance: nan
right of 
 Target variance: 0.4397678258744153 Landmark variance: 0.4418984124606306
left of 
 Target variance: nan Landmark variance: nan
